# MMC Implementation

## Correct Implementation

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import skmob
from tqdm import tqdm
import sys
sys.path.append('../')
from src.geo_utils import (get_clusters_from_tdf,
                            assign_tdf_points_to_clusters,
                            get_mmc_transitions,
                            get_stationary_vector)
import os
import pickle

In [2]:
from src.geo_utils import (analyze_dates_with_events,
                           analyze_number_of_events, 
                           analyze_number_of_events_per_day, 
                           analyze_space_drift, 
                           analyze_speed_drift,
                           analyze_time_drift,
                          get_mean_distance_bw_clusters)

In [3]:
pd.set_option('max_columns',50)
pd.set_option('max_rows',100)

In [4]:
warnings.filterwarnings('ignore')

In [5]:
def get_distance(row):
    lat, lng, lat_last, lng_last = row['lat'], row['lng'], row['lat_last'], row['lng_last']
    return skmob.utils.utils.distance((lat, lng), (lat_last, lng_last))

### Evaluation of Implementation

In [ ]:
# Reading all the DataFrames and consolidating it into one
data_files = sorted([x for x in os.listdir('../data/') if 'geo_' in x])

data = pd.DataFrame()
for data_file in tqdm(data_files):
    data_i = pd.read_csv(f'../data/{data_file}')
    data = data.append(data_i)
data = data.reset_index(drop=True)

In [ ]:
# Homogenizing the dataset format

geo_columns = ['user', 'hour', 'lat', 'lng']

data['hour'] = data['date'] + ' ' + data['time']
data['hour'] = pd.to_datetime(data['hour'])
data['user'] = data['user_id'].map(int)

data = data[geo_columns]
data = data.drop_duplicates()
data = data.groupby(['user','hour'], as_index=False).nth(0)
data = data.reset_index(drop=True)


# Estimating distance and time differences

data[['lat_last','lng_last']] = data.groupby('user').shift(1)[['lat','lng']]
data['seconds_diff'] = data.groupby('user')['hour'].diff(1).dt.seconds

data['distance_to_last_km'] = data.apply(get_distance, axis=1) 
data['speed_mps'] = data['distance_to_last_km'] / data['seconds_diff'] * 1000

In [ ]:
data.info()

In [ ]:
del data['lat_last'], data['lng_last']

In [ ]:
data.head()

- El 99% de los registros están distanciados entre 0 y 5 segundos

#### Saving Data

In [ ]:
# Saving Data
data.to_parquet('../data/geolife_consolidated.parquet', index=False)

#### Loading Data (backup)

In [6]:
# Loading Data
data = pd.read_parquet('../data/geolife_consolidated.parquet')

In [7]:
data['date'] = data['hour'].dt.date

In [8]:
def analyze_time_drift(data):
    '''Analiza la distribución de la variación de tiempo (segundos) entre evento y evento'''
    
    # Distribución de número de eventos
    serie_seconds_events = data['seconds_diff'].value_counts()
    serie_seconds_events_log = np.log10(serie_seconds_events)

    print(serie_seconds_events)
    print()
    print(serie_seconds_events.quantile([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1]))

    fig = plt.figure(figsize=(15,4))
    fig.add_subplot(1,2,1)
    data['seconds_diff'].hist(bins=100)
    plt.ylabel('Numero de Eventos')
    plt.xlabel('Delta de Segundos')
    
    fig.add_subplot(1,2,2)

    serie_seconds_events_log.hist(bins=30)
    plt.title('Distribución de eventos según drift en segundos (log10)');
    plt.ylabel('Numero de Eventos')
    plt.xlabel('Delta de Segundos (log10 scale)');
    

def analyze_space_drift(data):
    '''Analiza la distribución de la variación de distancia (Km) entre evento y evento'''

    serie_distance_events = data['distance_to_last_km'].value_counts()
    serie_distance_events_log = np.log10(serie_distance_events)
    
    print(serie_distance_events)
    print()
    print(serie_distance_events.quantile([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1]))

    fig = plt.figure(figsize=(15,4))
    fig.add_subplot(1,2,1)
    data['distance_to_last_km'].hist(bins=100)
    plt.ylabel('Numero de Eventos')
    plt.xlabel('Delta de KM');
    
    fig.add_subplot(1,2,2)

    serie_distance_events_log.hist(bins=30)
    plt.title('Distribución de eventos según drift de distancia (log10)');
    plt.ylabel('Numero de Eventos')
    plt.xlabel('Delta de KM (log10 scale)');
    
    
def analyze_speed_drift(data):
    '''Analiza la velocidad (metros por segundo) de cada evento'''

    serie_speed = data['speed_mps']
    serie_speed_log = np.log10(serie_speed+1)
    
    print(serie_speed.quantile([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1]))

    fig = plt.figure(figsize=(15,4))
    
    fig.add_subplot(1,2,1)
    serie_speed.hist(bins=100)
    plt.title('Distribución de eventos según velocidad');
    plt.ylabel('Numero de Eventos')
    plt.xlabel('Velocidad');
    
    fig.add_subplot(1,2,2)
    serie_speed_log.hist(bins=30)
    plt.title('Distribución de eventos según velocidad (log10)');
    plt.ylabel('Numero de Eventos')
    plt.xlabel('Velocidad (log10 scale)');
    
    
def analyze_number_of_events_per_day(data):
    '''Analiza la distribución de la variación de tiempo (segundos) entre evento y evento'''

    # Distribución de número de eventos
    df_events_per_day = data.groupby(['date','user'], as_index=False).size()
    
    serie_user_events_per_day = df_events_per_day['size']
    serie_user_events_per_day_log = np.log10(serie_user_events_per_day)

    print(serie_user_events_per_day)
    print()
    print(df_events_per_day['size'].quantile([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1]))

    fig = plt.figure(figsize=(15,4))

    fig.add_subplot(1,2,1)
    serie_user_events_per_day.hist(bins=30)
    plt.title('Distribución de número de eventos por usuarios por día');
    plt.ylabel('Numero de Usuarios')
    plt.xlabel('Número de eventos en un día')    
    
    fig.add_subplot(1,2,2)
    serie_user_events_per_day_log.hist(bins=30)
    plt.title('Distribución de número de eventos por usuarios por día (log10)');
    plt.ylabel('Numero de Usuarios')
    plt.xlabel('Número de eventos en un día (log10 scale)')
    
def analyze_number_of_events(data):
    # Distribución de número de eventos
    serie_user_events = data['user'].value_counts()
    serie_user_events_log = np.log10(serie_user_events)

    print(serie_user_events)
    print()
    print(serie_user_events.quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1]))

    fig = plt.figure(figsize=(15,4))

    fig.add_subplot(1,2,1)
    serie_user_events.hist(bins=30)
    plt.title('Distribución de usuarios según Nro de Eventos');
    plt.ylabel('Numero de Usuarios')
    plt.xlabel('Número de eventos')    
    
    fig.add_subplot(1,2,2)
    serie_user_events_log.hist(bins=30)
    plt.title('Distribución de usuarios según Nro de Eventos (log10)');
    plt.ylabel('Numero de Usuarios')
    plt.xlabel('Número de eventos (log10 scale)')
    

def analyze_dates_with_events(data):
    # Distribución de número de eventos
    serie_user_events = data[['user','date']].drop_duplicates()['user'].value_counts()
    serie_user_events_log = np.log10(serie_user_events)

    print(serie_user_events)

    fig = plt.figure(figsize=(15,4))
    fig.add_subplot(1,2,1)
    serie_user_events.hist(bins=30)
    plt.title('Distribución de usuarios según Nro de Días con Eventos');
    plt.ylabel('Numero de Usuarios')
    plt.xlabel('Número de Días con eventos')
    
    fig.add_subplot(1,2,2)

    serie_user_events_log.hist(bins=30)
    plt.title('Distribución de usuarios según Nro de Días con Eventos (log10)');
    plt.ylabel('Numero de Usuarios')
    plt.xlabel('Número de eventos (log10 scale)')

In [ ]:
analyze_number_of_events(data)

In [ ]:
analyze_number_of_events_per_day(data)

In [ ]:
analyze_dates_with_events(data)

In [ ]:
analyze_space_drift(data)

In [ ]:
analyze_space_drift(data)

In [ ]:
analyze_speed_drift(data)

#### Latitud y Longitud

In [ ]:
data['lat'].hist(bins=50)

In [ ]:
data['lng'].hist(bins=50)

In [ ]:
data.shape

- Datashader

In [ ]:
dd = data.sample(20000)

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot(data=dd, x='lng', y='lat')

In [ ]:
sns.kdeplot(data=dd, x='lat', y='lng', fill=True)

__Escenarios:__
- Base
- Filtro por país
- Filtro por tiempo en lugar

__Acuerdos__
- Quitar usuarios con menos de 1000 eventos
- Probar los 3 escenarios

In [9]:
def filter_min_events_per_user(_df, min_events, user_col = 'user'):
    print('Datos iniciales:')
    print('# Registros: {:,}'.format(_df.shape[0]), '\t|', '# Usuarios: {}'.format(_df[user_col].nunique()))
    print('-'*70)
    users_to_del = _df.groupby(user_col)[user_col].count()[_df.groupby(user_col)[user_col].count()<min_events].index
    _df = _df[~_df[user_col].isin(users_to_del)]
    
    print('Datos finales:')
    print('# Registros: {:,}'.format(_df.shape[0]), '\t|', '# Usuarios: {}'.format(_df[user_col].nunique()))
    print('-'*70)
    return _df 

In [10]:
data = filter_min_events_per_user(data, 1000)

Datos iniciales:
# Registros: 24,178,078 	| # Usuarios: 182
----------------------------------------------------------------------
Datos finales:
# Registros: 24,167,443 	| # Usuarios: 158
----------------------------------------------------------------------


In [11]:
def filter_min_days_per_user(_df, min_days, user_col = 'user', date_col  = 'date'):
    print('Datos iniciales:')
    print('# Registros: {:,}'.format(_df.shape[0]), '\t|', '# Usuarios: {}'.format(_df[user_col].nunique()))
    print('-'*70)
    users_to_del = _df.groupby(user_col)[date_col].nunique()[_df.groupby(user_col)[date_col].nunique()<min_days].index
    _df = _df[~_df[user_col].isin(users_to_del)]
    
    print('Datos finales:')
    print('# Registros: {:,}'.format(_df.shape[0]), '\t|', '# Usuarios: {}'.format(_df[user_col].nunique()))
    print('-'*70)
    return _df 

In [12]:
data = filter_min_days_per_user(data, 10)

Datos iniciales:
# Registros: 24,167,443 	| # Usuarios: 158
----------------------------------------------------------------------
Datos finales:
# Registros: 23,576,497 	| # Usuarios: 120
----------------------------------------------------------------------


In [ ]:
analyze_number_of_events_per_day(data)

In [13]:
from skmob.preprocessing import (filtering, 
                                 detection, 
                                 compression, 
                                 clustering)

In [ ]:
# Creating a Trajectory DF to analyze the impact of filtering by velocity
trgeo = skmob.TrajDataFrame(
    data[data['user']==1], 
    datetime='hour',
    user_id='user'
)

In [ ]:
del trgeo
gc.collect()

In [ ]:
#Aproximadamente reducir a 10%

In [14]:
import datetime

In [15]:
def get_mmc_clusters_stavectors(geo):
    '''Get MMC Clusters and Stationary Vectors
    
    Parameters
    ----------
        geo (pd.Dataframe): Dataframe of 1 user_id. Must contain only the columns: ['user', 'hour', 'lat', 'lng']
    
    Returns
    -------
        clusters (pd.Dataframe): Dataframe with the clusters and the stationary value
        m (folium.folium.Map): Map generated with the clusters on it.
    '''
    # TDF Definition
    trgeo = skmob.TrajDataFrame(
        geo, 
        datetime='hour',
        user_id='user'
    )

    # Cluster Generation
    clusters, m = get_clusters_from_tdf(
        trgeo,
        verbose=True,
        max_speed_kmh= 0.1,
        detect_stops=False,
        compress=False ,
        minutes_for_a_stop=2,
        spatial_radius_km=0.2,
        spatial_radius_compress_km=.2,
        cluster_radius_km=0.5,
        min_samples=2
    )

    # Cluster Assignation
    trgeo_cl, distances = assign_tdf_points_to_clusters(
        tdf=trgeo, 
        clusters=clusters
    )

    # Generation of Transit Dataframe
    transit_df = get_mmc_transitions(trgeo_cl)

    # Generation of Transit Matrix
    #transit_df = transit_df[transit_df['cluster'].isin(transit_df['cluster_next'].unique())]
    #transit_df = transit_df[transit_df['cluster_next'].isin(transit_df['cluster'].unique())]
    display(transit_df)
    
    #Lógica adicional para añadir entradas o salidas y hacer simétrica la matriz
    
    #Registros cuyos clusters nunca aparecen como salida
    cluster_mode = transit_df['cluster_next'].mode()[0] # Cluster más frecuente
    df_never_next= transit_df[~transit_df['cluster'].isin(transit_df['cluster_next'].unique())]
    df_never_next['cluster_next'] = df_never_next['cluster']
    df_never_next['cluster'] = cluster_mode
    if(df_never_next.shape[0]>0):
        print('\nMostrando registros ficticios ')
        df_never_next['datetime'] = df_never_next['datetime']  - datetime.timedelta(milliseconds = 10)
        display(df_never_next)
        
        transit_df = pd.concat([transit_df,df_never_next], ignore_index=True ).sort_values(by='datetime')
        
    #Registros cuyos clusters nunca aparecen como inicio
    df_never_ini= transit_df[~transit_df['cluster_next'].isin(transit_df['cluster'].unique())]
    df_never_ini['cluster'] = df_never_ini['cluster_next']
    df_never_ini['cluster_next'] = cluster_mode
    if(df_never_ini.shape[0]>0):
        print('\nMostrando registros ficticios')
        df_never_ini['datetime'] = df_never_ini['datetime']  + datetime.timedelta(milliseconds = 10)
        
        display(df_never_ini)
        transit_df = pd.concat([transit_df,df_never_ini], ignore_index=True ).sort_values(by='datetime')
    
    #transit_df.loc[transit_df['cluster_next'].isnull(),'cluster_next'] = 0
    transit_matrix = pd.crosstab(transit_df['cluster'], 
                                 transit_df['cluster_next'],
                                 normalize='index')
    display(transit_matrix)


    



    
    return clusters, m, transit_matrix, transit_df
    

#Versión antigua
def get_mmc_clusters_stavectors(geo):
    '''Get MMC Clusters and Stationary Vectors
    
    Parameters
    ----------
        geo (pd.Dataframe): Dataframe of 1 user_id. Must contain only the columns: ['user', 'hour', 'lat', 'lng']
    
    Returns
    -------
        clusters (pd.Dataframe): Dataframe with the clusters and the stationary value
        m (folium.folium.Map): Map generated with the clusters on it.
    '''
    # TDF Definition
    trgeo = skmob.TrajDataFrame(
        geo, 
        datetime='hour',
        user_id='user'
    )

    # Cluster Generation
    clusters, m = get_clusters_from_tdf(
        trgeo,
        verbose=True,
        max_speed_kmh= 0.1,
        detect_stops=False,
        compress=False ,
        minutes_for_a_stop=2,
        spatial_radius_km=0.2,
        spatial_radius_compress_km=.2,
        cluster_radius_km=0.5,
        min_samples=2
    )

    # Cluster Assignation
    trgeo_cl, distances = assign_tdf_points_to_clusters(
        tdf=trgeo, 
        clusters=clusters
    )

    # Generation of Transit Dataframe
    transit_df = get_mmc_transitions(trgeo_cl)

    # Generation of Transit Matrix
    #transit_df = transit_df[transit_df['cluster'].isin(transit_df['cluster_next'].unique())]
    #transit_df = transit_df[transit_df['cluster_next'].isin(transit_df['cluster'].unique())]
    display(transit_df)
    transit_df.loc[transit_df['cluster_next'].isnull(),'cluster_next'] = 0
    transit_matrix = pd.crosstab(transit_df['cluster'], 
                                 transit_df['cluster_next'],
                                 normalize='index').values
    display(pd.DataFrame(transit_matrix))
    

    #Lógica adicional para remover clusters sin salida o entrada
    
    
    
    
    
    
    ##Retiramos las filas que tienen solo ceros y una columna con 1
    if (transit_matrix.shape[0]>2 ):
        print('Revisando filas de la Matriz de transición...')
        print('mostrando clusters iniciales')
        display(clusters)
        rows_to_drop = (transit_matrix==0).sum(axis=1)==(transit_matrix.shape[1]-1)#todos los valores en 0 excepto 1
        rows_to_drop = [idx for idx,row in enumerate(rows_to_drop) if row]
        
        if len(rows_to_drop)>0:
            for row in rows_to_drop:
                print('Eliminando la fila {}'.format(row))
                if(row < transit_matrix.shape[0]):
                    transit_matrix = np.delete(transit_matrix,row,0)
                    print('{} eliminada de la fila de la transit_matrix')
                if(row < transit_matrix.shape[1]):
                    transit_matrix = np.delete(transit_matrix,row,1)
                    print('{} eliminada de la Columna de la transit_matrix')

                    
            print('Transit matrix actual:')
            display(pd.DataFrame(transit_matrix))

            clusters=clusters.drop(rows_to_drop, axis=0  )
            print('clusters {} eliminados'.format(rows_to_drop))

    if(transit_matrix.shape[1]>2):    
        ##Retiramos las columnas que tienen solo ceros una fila con 1
        print('Revisando columnas de la Matriz de transición...')
        print('mostrando clusters iniciales')
        display(clusters)
        cols_to_drop = (transit_matrix==0).sum(axis=0)==(transit_matrix.shape[0]-1)
        cols_to_drop = [idx for idx,col in enumerate(cols_to_drop) if col]
        print(cols_to_drop)
        
        if len(cols_to_drop)>0: 
            for col in cols_to_drop:
                if(col < transit_matrix.shape[0]):
                    transit_matrix = np.delete(transit_matrix,col,0)
                    print('{} eliminada de la fila de la transit_matrix')
                if(col < transit_matrix.shape[1]):
                    transit_matrix = np.delete(transit_matrix,col,1)
                    print('{} eliminada de la Columna de la transit_matrix')

            clusters=clusters[~clusters['cluster'].isin(cols_to_drop)]

            print('clusters {} eliminados'.format(cols_to_drop))
            print('Transit matrix actual:')
            display(pd.DataFrame(transit_matrix))



    
    return clusters, m, transit_matrix, transit_df

In [16]:
data['user'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  22,  23,  24,  25,  26,
        28,  29,  30,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  46,  50,  51,  52,  55,  56,  57,  58,  61,  62,
        63,  64,  65,  66,  67,  68,  71,  73,  74,  75,  78,  79,  81,
        82,  83,  84,  85,  88,  89,  91,  92,  93,  94,  95,  96,  97,
        99, 101, 102, 103, 104, 110, 111, 112, 113, 114, 115, 119, 122,
       125, 126, 128, 130, 131, 134, 135, 136, 138, 139, 140, 141, 142,
       144, 147, 150, 153, 154, 155, 157, 158, 159, 163, 165, 167, 168,
       169, 174, 179])

In [17]:
# Extraction of clusters top 10
geo_clusters = {}
geo_clusters_img = {}
geo_clusters_transit = {}
geo_clusters_transit_df = {}
users = sorted(data['user'].unique())
for user in tqdm(users):
    print('Procesando user {}'.format(user))
    try: 
        geo = data[data['user']==user].reset_index(drop=True).copy()

        clusters, m, transit_matrix, transit_df = get_mmc_clusters_stavectors(geo)

        geo_clusters[user] = clusters
        geo_clusters_img[user] = m
        geo_clusters_transit[user] = transit_matrix
        geo_clusters_transit_df[user] = transit_df
        print('Procesamiento del user {} TERMINADO...'.format(user))
        print('-'*100 + '\n')
    except:
        print('Error en la generación de clusters del user {}'.format(user))

  0%|                                                   | 0/120 [00:00<?, ?it/s]

Procesando user 0
INFO: Noise Filtering applied
INFO: Clusters generated
(173870, 5) (379, 5) (379, 5) (379, 5) (379, 6)
INFO: 12 clusters generated.



100%|███████████████████████████████████████████| 12/12 [00:33<00:00,  2.77s/it]


,uid,datetime,lat,lng,date,cluster,cluster_next,transition
14,0,2008-10-23 02:54:15,39.984523,116.316188,2008-10-23,d_cl_11,d_cl_10,d_cl_11-d_cl_10
1874,0,2008-10-26 14:57:27,39.911518,116.304580,2008-10-26,d_cl_10,d_cl_08,d_cl_10-d_cl_08
3171,0,2008-10-28 02:37:21,40.004826,116.305323,2008-10-28,d_cl_08,d_cl_05,d_cl_08-d_cl_05
3506,0,2008-10-29 09:37:08,39.985720,116.326921,2008-10-29,d_cl_05,d_cl_09,d_cl_05-d_cl_09
3626,0,2008-10-29 09:46:43,39.966701,116.327688,2008-10-29,d_cl_09,d_cl_08,d_cl_09-d_cl_08
5630,0,2008-11-04 03:11:48,40.005072,116.305368,2008-11-04,d_cl_08,d_cl_05,d_cl_08-d_cl_05
6088,0,2008-11-10 03:40:42,39.989265,116.327001,2008-11-10,d_cl_05,d_cl_08,d_cl_05-d_cl_08
7428,0,2008-11-11 02:19:44,40.005079,116.305286,2008-11-11,d_cl_08,d_cl_00,d_cl_08-d_cl_00
8148,0,2008-11-12 04:33:12,40.001598,116.323275,2008-11-12,d_cl_00,d_cl_07,d_cl_00-d_cl_07
8729,0,2008-11-13 14:01:26,40.015449,116.324462,2008-11-13,d_cl_07,d_cl_00,d_cl_07-d_cl_00


cluster_next,d_cl_00,d_cl_01,d_cl_02,d_cl_03,d_cl_04,d_cl_05,d_cl_06,d_cl_07,d_cl_08,d_cl_09,d_cl_10,d_cl_11
cluster,,,,,,,,,,,,
d_cl_00,0.000000,0.043478,0.26087,0.000000,0.043478,0.304348,0.043478,0.130435,0.000000,0.000000,0.000000,0.173913
d_cl_01,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
d_cl_02,0.500000,0.125000,0.00000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.250000,0.000000,0.000000
d_cl_03,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
d_cl_04,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
d_cl_05,0.500000,0.000000,0.00000,0.000000,0.000000,0.000000,0.277778,0.000000,0.055556,0.166667,0.000000,0.000000
d_cl_06,0.066667,0.000000,0.00000,0.466667,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333
d_cl_07,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
d_cl_08,0.250000,0.000000,0.25000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


...


100%|███████████████████████████████████████████| 14/14 [00:35<00:00,  2.51s/it]


,uid,datetime,lat,lng,date,cluster,cluster_next,transition
330,179,2008-08-20 12:23:39,39.975230,116.329885,2008-08-20,d_cl_13,d_cl_11,d_cl_13-d_cl_11
863,179,2008-08-20 12:48:39,40.002971,116.331276,2008-08-20,d_cl_11,d_cl_09,d_cl_11-d_cl_09
909,179,2008-08-20 12:50:11,40.015456,116.321900,2008-08-20,d_cl_09,d_cl_03,d_cl_09-d_cl_03
927,179,2008-08-20 12:50:47,40.020450,116.318559,2008-08-20,d_cl_03,d_cl_07,d_cl_03-d_cl_07
952,179,2008-08-20 12:51:37,40.028060,116.315582,2008-08-20,d_cl_07,d_cl_06,d_cl_07-d_cl_06
...,...,...,...,...,...,...,...,...
149888,179,2008-11-17 06:45:42,40.034656,116.312820,2008-11-17,d_cl_06,d_cl_04,d_cl_06-d_cl_04
150189,179,2008-11-17 07:00:34,40.070180,116.311012,2008-11-17,d_cl_04,d_cl_01,d_cl_04-d_cl_01
150297,179,2008-11-17 07:04:10,40.081020,116.302700,2008-11-17,d_cl_01,d_cl_00,d_cl_01-d_cl_00
150602,179,2008-11-17 07:14:32,40.087012,116.304162,2008-11-17,d_cl_00,d_cl_04,d_cl_00-d_cl_04


cluster_next,d_cl_00,d_cl_01,d_cl_02,d_cl_03,d_cl_04,d_cl_05,d_cl_06,d_cl_07,d_cl_08,d_cl_09,d_cl_10,d_cl_11,d_cl_12,d_cl_13
cluster,,,,,,,,,,,,,,
d_cl_00,0.000000,0.540541,0.027027,0.000000,0.081081,0.00,0.000000,0.000000,0.189189,0.000000,0.00,0.054054,0.054054,0.054054
d_cl_01,0.562500,0.000000,0.000000,0.000000,0.395833,0.00,0.000000,0.000000,0.020833,0.000000,0.00,0.020833,0.000000,0.000000
d_cl_02,0.050000,0.000000,0.000000,0.050000,0.150000,0.05,0.000000,0.000000,0.000000,0.200000,0.05,0.350000,0.000000,0.100000
d_cl_03,0.000000,0.000000,0.027027,0.000000,0.000000,0.00,0.000000,0.621622,0.000000,0.351351,0.00,0.000000,0.000000,0.000000
d_cl_04,0.076923,0.519231,0.038462,0.000000,0.000000,0.00,0.288462,0.019231,0.000000,0.000000,0.00,0.038462,0.000000,0.019231
d_cl_05,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1.000000,0.000000,0.00,0.000000,0.000000,0.000000
d_cl_06,0.000000,0.000000,0.052632,0.000000,0.605263,0.00,0.000000,0.315789,0.000000,0.000000,0.00,0.000000,0.000000,0.026316
d_cl_07,0.000000,0.000000,0.000000,0.351351,0.027027,0.00,0.621622,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
d_cl_08,0.000000,0.000000,0.000000,0.000000,0.100000,0.00,0.000000,0.000000,0.000000,0.000000,0.90,0.000000,0.000000,0.000000


100%|██████████████████████████████████████| 120/120 [3:22:59<00:00, 101.50s/it]

Procesamiento del user 179 TERMINADO...
----------------------------------------------------------------------------------------------------



In [18]:
import pickle

In [19]:
with open('../data/geo_clusters', 'wb') as f:
    pickle.dump(geo_clusters,f)

In [20]:
with open('../data/geo_clusters_transit', 'wb') as f:
    pickle.dump(geo_clusters_transit,f)

In [21]:
with open('../data/geo_clusters_transit_df', 'wb') as f:
    pickle.dump(geo_clusters_transit_df,f)

In [ ]:
#Continuacion

In [ ]:
#Read data geo_clusters
with open(r"../data/geo_clusters", "rb") as input_file:
    geo_clusters = pickle.load(input_file)

In [ ]:
#Read data geo_clusters_transit
with open(r"../data/geo_clusters_transit", "rb") as input_file:
    geo_clusters_transit = pickle.load(input_file)

In [ ]:
#Read data geo_clusters
with open(r"../data/geo_clusters_transit_df", "rb") as input_file:
    geo_clusters_transit_df = pickle.load(input_file)

In [ ]:
get_stationary_vector(geo_clusters_transit[0] )

In [ ]:
pd.DataFrame(geo_clusters_transit[0])

In [ ]:
geo_clusters[0]

In [ ]:
geo_clusters_transit[1]

In [25]:
get_stationary_vector(geo_clusters_transit[1] )

Shape of transitMatrix:  (5, 5)


array([[0.39285714],
       [0.32142857],
       [0.21428571],
       [0.05357143],
       [0.01785714]])

In [ ]:
geo_clusters[1]['sta_vector'] = get_stationary_vector(geo_clusters_transit[1] )

In [ ]:
geo_clusters[1]['sta_vector']

In [ ]:
geo_clusters_transit[1]

In [29]:
geo_clusters_transit[103]

cluster_next,d_cl_00,d_cl_02,d_cl_03,d_cl_04,d_cl_05,d_cl_06,d_cl_07
cluster,,,,,,,
d_cl_00,0.000000,0.2,0.200000,0.0,0.60,0.000000,0.000000
d_cl_02,0.500000,0.0,0.000000,0.0,0.50,0.000000,0.000000
d_cl_03,0.000000,0.0,0.000000,0.0,0.25,0.666667,0.083333
d_cl_04,0.000000,0.0,1.000000,0.0,0.00,0.000000,0.000000
d_cl_05,0.222222,0.0,0.666667,0.0,0.00,0.111111,0.000000
d_cl_06,0.300000,0.1,0.300000,0.1,0.20,0.000000,0.000000
d_cl_07,0.000000,0.0,0.000000,0.0,0.00,1.000000,0.000000


In [35]:
get_stationary_vector(geo_clusters_transit[103] )

Shape of transitMatrix:  (7, 7)


array([[0.14952781],
       [0.05351522],
       [0.28016789],
       [0.02360965],
       [0.23373557],
       [0.23609654],
       [0.02334732]])

In [36]:
geo_clusters[103]

,cluster,lat,lng
0,0,39.930145,116.298312
1,1,39.928991,116.314754
2,2,39.972442,116.300565
3,3,39.958895,116.349013
4,4,39.947488,116.371845
5,5,39.945318,116.350602
6,6,39.966870,116.348727
7,7,39.975517,116.330692


In [37]:
geo_clusters_transit[103]

cluster_next,d_cl_00,d_cl_02,d_cl_03,d_cl_04,d_cl_05,d_cl_06,d_cl_07
cluster,,,,,,,
d_cl_00,0.000000,0.2,0.200000,0.0,0.60,0.000000,0.000000
d_cl_02,0.500000,0.0,0.000000,0.0,0.50,0.000000,0.000000
d_cl_03,0.000000,0.0,0.000000,0.0,0.25,0.666667,0.083333
d_cl_04,0.000000,0.0,1.000000,0.0,0.00,0.000000,0.000000
d_cl_05,0.222222,0.0,0.666667,0.0,0.00,0.111111,0.000000
d_cl_06,0.300000,0.1,0.300000,0.1,0.20,0.000000,0.000000
d_cl_07,0.000000,0.0,0.000000,0.0,0.00,1.000000,0.000000


In [38]:
geo_clusters_transit_df[103]

,uid,datetime,lat,lng,date,cluster,cluster_next,transition
39,103,2008-08-27 06:01:30.990,39.975095,116.332818,2008-08-27,d_cl_03,d_cl_07,d_cl_07-d_cl_06
0,103,2008-08-27 06:01:31.000,39.975095,116.332818,2008-08-27,d_cl_07,d_cl_06,d_cl_07-d_cl_06
1,103,2008-08-27 09:30:47.000,39.965470,116.347378,2008-08-27,d_cl_06,d_cl_03,d_cl_06-d_cl_03
2,103,2008-08-27 09:33:19.000,39.957665,116.347372,2008-08-27,d_cl_03,d_cl_05,d_cl_03-d_cl_05
3,103,2008-08-27 23:56:41.000,39.946928,116.350065,2008-08-27,d_cl_05,d_cl_03,d_cl_05-d_cl_03
4,103,2008-08-28 00:02:49.000,39.960468,116.348695,2008-08-28,d_cl_03,d_cl_06,d_cl_03-d_cl_06
5,103,2008-08-28 00:05:14.000,39.968635,116.348302,2008-08-28,d_cl_06,d_cl_00,d_cl_06-d_cl_00
6,103,2008-08-28 23:35:56.000,39.931352,116.299968,2008-08-28,d_cl_00,d_cl_05,d_cl_00-d_cl_05
7,103,2008-08-29 00:08:00.000,39.947095,116.350647,2008-08-29,d_cl_05,d_cl_03,d_cl_05-d_cl_03
8,103,2008-08-29 00:14:17.000,39.960608,116.349043,2008-08-29,d_cl_03,d_cl_06,d_cl_03-d_cl_06


In [26]:
for user in tqdm(geo_clusters):
    geo_clusters[user]['sta_vector'] = get_stationary_vector(geo_clusters_transit[user] )
    print('sta_vector for user {} created...'.format(user))

 68%|████████████████████████████             | 80/117 [00:00<00:00, 417.91it/s]


Shape of transitMatrix:  (12, 12)
sta_vector for user 0 created...
Shape of transitMatrix:  (5, 5)
sta_vector for user 1 created...
Shape of transitMatrix:  (8, 8)
sta_vector for user 2 created...
Shape of transitMatrix:  (13, 13)
sta_vector for user 3 created...
Shape of transitMatrix:  (7, 7)
sta_vector for user 4 created...
Shape of transitMatrix:  (2, 2)
sta_vector for user 5 created...
Shape of transitMatrix:  (4, 4)
sta_vector for user 6 created...
Shape of transitMatrix:  (3, 3)
sta_vector for user 7 created...
Shape of transitMatrix:  (7, 7)
sta_vector for user 8 created...
Shape of transitMatrix:  (2, 2)
sta_vector for user 9 created...
Shape of transitMatrix:  (35, 35)
sta_vector for user 10 created...
Shape of transitMatrix:  (15, 15)
sta_vector for user 11 created...
Shape of transitMatrix:  (6, 6)
sta_vector for user 12 created...
Shape of transitMatrix:  (13, 13)
sta_vector for user 13 created...
Shape of transitMatrix:  (24, 24)
sta_vector for user 14 created...
Shape of

ValueError: Length of values (7) does not match length of index (8)

In [ ]:
geo_clusters[0]

In [ ]:
def get_mmc_distances_matrix(mmc_list_a, mmc_list_b):
    '''Get MMC Distances Matrix
    
    It estimates the distances of the elements of two lists of MMCs.
    Afterwards, it stores the distances in a matrix.
    
    Parameters
    ----------
        mmc_list_a (list): List of MMCs A
        mmc_list_b (list): List of MMCs B

    Returns
    -------
        distance_matrix (np.array): Matrix with all the distances calculated.
    
    '''
    n = len(mmc_list_a)
    m = len(mmc_list_b)
    distance_matrix = np.zeros((n, m))
    for i, mmc_i in enumerate(mmc_list_a):
        for j, mmc_j in enumerate(mmc_list_b):
            distance = get_mean_distance_bw_clusters(mmc_i, mmc_j)
            distance_matrix[i,j] = round(distance,4)
            
    return distance_matrix

In [ ]:
geo_clusters[0]

In [ ]:
dm  = get_mmc_distances_matrix(list(geo_clusters.values()),list(geo_clusters.values()))

In [ ]:
# Extraction of clusters 
geo_clusters = {}
geo_clusters_img = {}
geo_clusters_transit = {}
geo_clusters_transit_df = {}
users = sorted(data['user'].unique())
for user in tqdm(users):

    geo = data[data['user']==user].reset_index(drop=True).copy()

    clusters, m, transit_matrix, transit_df = get_mmc_clusters_stavectors(geo)

    geo_clusters[user] = clusters
    geo_clusters_img[user] = m
    geo_clusters_transit[user] = transit_matrix
    geo_clusters_transit_df[user] = transit_df


In [ ]:
# Creating a Trajectory DF to analyze the impact of filtering by velocity
trgeo = skmob.TrajDataFrame(
    data[data['user']==10], 
    datetime='hour',
    user_id='user'
)

In [ ]:
# 1. Noise Filtering
print(trgeo.shape)
trgeo = filtering.filter(trgeo, max_speed_kmh=0.1)
print(trgeo.shape)

In [ ]:
for user in data[data['user']<10]['user'].unique():
    trgeo = skmob.TrajDataFrame(data[data['user']==user], datetime='hour',user_id='user')
    val1 = trgeo.shape[0]
    trgeo = filtering.filter(trgeo, max_speed_kmh=0.2)
    val2 = trgeo.shape[0]
    print('Puntos reducidos de {:,} a {:,}'.format(val1,val2))

In [ ]:
3305 / 108607

In [ ]:
print('a')

In [ ]:
aux_dict = {}
aux_dict['user'] = []
aux_dict['number_points'] = []
aux_dict['min_tmstmp'] = []
aux_dict['max_tmstmp'] = []
for usr in data['user'].unique():
    aux_dict['user'].append(usr)
    aux_dict['number_points'].append(data[data['user']==usr].shape[0])
    aux_dict['min_tmstmp'].append(data[data['user']==usr]['hour'].min())
    aux_dict['max_tmstmp'].append(data[data['user']==usr]['hour'].max())
aux_dict = pd.DataFrame(aux_dict)
aux_dict['duracion'] = (aux_dict['max_tmstmp'] - aux_dict['min_tmstmp']).dt.total_seconds()/ 60
aux_dict.sort_values(by='duracion', ascending=True, inplace=True)
display(aux_dict)

In [ ]:
fig,ax = plt.subplots(figsize=(6,4))
ax.set_xscale('log')
plt.scatter(aux_dict['duracion'].values, aux_dict['number_points'].values, color='r')
plt.plot(aux_dict['duracion'].values, aux_dict['number_points'].values , color='r', linestyle='--')
"""
plt.axvline(60,0)
plt.text(60,0, 'H')
plt.axvline(60*24,0)
plt.text(60*24,0, 'D')
"""
plt.axvline(60*24*7,0)
plt.text(60*24*7,0, 'S')
plt.axvline(60*24*7*30,0)
plt.text(60*24*7*30,0, 'M')
ax.set_xlabel('Tiempo en minutos - log')
ax.set_ylabel('Cantidad de puntos')
#plt.axvline(60*24*7*30*365,0)
#plt.text(60*24*7*30*365,0, 'Y')
#plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(12,6))
#ax.set_xscale('log')
plt.scatter(aux_dict['duracion'].values, aux_dict['number_points'].values)
plt.plot(aux_dict['duracion'].values, aux_dict['number_points'].values )

#plt.show()

In [ ]:
data['seconds_diff'].nlargest(30)

-------|-----|-----------|---------|

--xxXx--|--xX-x--|--x--xX--|

In [ ]:
data['distance_to_last_km'].nlargest(30)